In [18]:
import requests
from bs4 import BeautifulSoup as BS
import json
import numpy as np
import pandas as pd
import csv
import ssl
import urllib.request 
import urllib.parse
import urllib.error
import re
import time
from lxml import etree
import lxml.html as PARSER
from urllib.request import Request, urlopen

In [19]:
req_headers = {
    'accept': '/',
    'accept-encoding': 'gzip, deflate, br',
    'user-agent': 'PostmanRuntime/7.28.4'
}

# For ignoring SSL certificate errors
ctx = ssl.create_default_context()
ctx.check_hostname = False
ctx.verify_mode = ssl.CERT_NONE

In [21]:
domain = "https://www.redfin.com"
addresses_list = []
sold_prices_list = []
bed_list = []
bath_list = []
sqft_list = []
hoa_list = []
yearBuild_list = []
type_list = []
lotSize_list = []
sold_date_main_list = []

# Location
fremont = '6671/CA/Fremont'
losAngeles = '11203/CA/Los-Angeles'
seattle = '16163/WA/Seattle'
newYork = '30749/NY/New-York'

def isXpath(content, xpath):
    root = PARSER.fromstring(content)
    if root.xpath(xpath):
        return True
    return False

for page in range(1,10):

    # Sold Properties
    soldPropertiesURL = 'https://www.redfin.com/city/' + fremont + '/filter/property-type=house+condo+townhouse+multifamily,include=sold-3mo/page-' + str(page)
    time.sleep(np.random.lognormal(0,1))
    req = Request(soldPropertiesURL, headers={'User-Agent': 'Mozilla/5.0'})
    webpage = urlopen(req).read()
    text = webpage.decode(encoding="utf8", errors="ignore")
        
    soup = BS(text, 'html.parser')
    
    bottomV2_list  = soup.findAll('div',attrs={'class':"bottomV2"})
    id_list  = re.findall('href=\"(.*?)\"',str(bottomV2_list))
    sold_date_list = soup.find_all("div", attrs={'class':["Pill Pill--sold padding-vert-smallest padding-horiz-smaller font-size-smaller font-weight-bold font-color-white HomeSash margin-top-smallest margin-right-smaller", "Pill Pill--red padding-vert-smallest padding-horiz-smaller font-size-smaller font-weight-bold font-color-white HomeSash margin-top-smallest margin-right-smaller"]})
    
    for i in range(len(sold_date_list)):
        sold_date_list[i] = sold_date_list[i].get_text().replace("SOLD BY REDFIN ","").replace("BOUGHT WITH REDFIN ","").replace("SOLD ","")
        sold_date_list[i] = str(sold_date_list[i])
    
    sold_date_main_list.extend(sold_date_list)
    
    # Construct Single Property URL
    for i in range(len(bottomV2_list)):
        singlePropertyURL = domain + id_list[i]
        time.sleep(np.random.lognormal(0,1))
        singlePropertyReq = Request(singlePropertyURL, headers={'User-Agent': 'Mozilla/5.0'})
        singlePropertyWebpage = urlopen(singlePropertyReq).read()
        singlePropertyText = singlePropertyWebpage.decode(encoding="utf8", errors="ignore")
        singlePropertySoup = BS(singlePropertyText, 'html.parser')
        dom = etree.HTML(str(singlePropertySoup))
    
        # Locate element
        address_list = singlePropertySoup.findAll('div',attrs={'class':"street-address"})
        city_list = singlePropertySoup.findAll('div',attrs={'class':"dp-subtext"})
        address = address_list[0].get_text() + city_list[0].get_text()
        print("Reading Property Info: " + address)

        sold_price_list = singlePropertySoup.findAll('div',attrs={'class':"statsValue"})
        bed = dom.xpath('//div[@class="stat-block beds-section"][2]/div/text()')
        bath = dom.xpath('//div[@class="stat-block baths-section"][1]/div/text()')
        sqft = dom.xpath('//div[@class="stat-block sqft-section"][1]/span/text()')
        lotSize = dom.xpath('//span[text() = "Lot Size"]/../span[2]/text()')
        if isXpath(singlePropertyText, '//span[text() = "HOA Dues"]'):
            hoa = dom.xpath('//span[text() = "HOA Dues"]/../span[2]/text()')
        else:
            hoa = 0
        type = dom.xpath('//span[text() = "Style"]/../div/text()')[0]
        yearBuild = dom.xpath('//span[text() = "Year Built"]/../div/text()')[0]
    
        address = address_list[0].get_text() + city_list[0].get_text()
        sold_price = sold_price_list[0].get_text()
        
        # Fix types
        bed = 0 if len(bed)==0 else bed[0]
        bath = 0 if len(bath)==0 else bath[0]
        sqft = 0 if len(sqft)==0 else sqft[0]
        lotSize = 0 if len(lotSize)==0 else lotSize[0]
        if hoa != 0:
            if len(hoa)!=0:
                hoa = hoa[0].replace("/month","")
            else:
                hoa = 0

        # add to main list
        addresses_list.append(address)
        sold_prices_list.append(sold_price)
        bed_list.append(bed)
        bath_list.append(bath)
        sqft_list.append(sqft)
        hoa_list.append(hoa)
        yearBuild_list.append(yearBuild)
        type_list.append(type)
        lotSize_list.append(lotSize)

df = pd.DataFrame(list(zip(addresses_list, sold_prices_list, sold_date_main_list, bed_list, bath_list, sqft_list,
                           lotSize_list, hoa_list, yearBuild_list, type_list)),
                  columns=("Address", "Sold", "Sold Date", "Bed", "Bath", "Size", 
                           "Lot Size", "HOA", "Year Build", "Type"))
df.index+=1
df.to_csv('redfinOutput.csv')
df

Reading Property Info: 62 Shaniko Cmn,Fremont, CA 94539
Reading Property Info: 45552 Cheyenne Pl,Fremont, CA 94539
Reading Property Info: 1201 Valdez Way,FREMONT, CA 94539
Reading Property Info: 39762 Blacow Rd,Fremont, CA 94538
Reading Property Info: 4405 Calypso Ter,Fremont, CA 94555
Reading Property Info: 35211 Santiago St,Fremont, CA 94536
Reading Property Info: 47562 Minaret Falls Ter,Fremont, CA 94539
Reading Property Info: 168 Staircase Falls Cmn,Fremont, CA 94539
Reading Property Info: 196 Carson Falls Ter,Fremont, CA 94539
Reading Property Info: 38781 Stonington Ter,FREMONT, CA 94536
Reading Property Info: 2401 Corriea Way,FREMONT, CA 94539
Reading Property Info: 2500 Tassajara Ter,Fremont, CA 94538
Reading Property Info: 35451 Cabrillo Dr,Fremont, CA 94551
Reading Property Info: 5157 Waller Ave,Fremont, CA 94536
Reading Property Info: 1750 Radiance Cmn Unit 301 F9,Fremont, CA 94539
Reading Property Info: 701 Boar Cir,Fremont, CA 94539
Reading Property Info: 4649 Victoria Park

Reading Property Info: 4463 Hyde Cmn #302,Fremont, CA 94538
Reading Property Info: 5417 Diana Cmn,FREMONT, CA 94555
Reading Property Info: 4874 Rogers Ave,Fremont, CA 94536
Reading Property Info: 875 Nina Way,Fremont, CA 94539
Reading Property Info: 4649 Richmond Ave,Fremont, CA 94536
Reading Property Info: 41415 Millenium Ter,FREMONT, CA 94538
Reading Property Info: 35815 Hibiscus Ct,Fremont, CA 94536
Reading Property Info: 35623 Chaplin Dr,Fremont, CA 94536
Reading Property Info: 5387 Edmonton Cmn,Fremont, CA 94555
Reading Property Info: 45128 Warm Springs Blvd #656,Fremont, CA 94539-9453
Reading Property Info: 42321 Paich Ct,Fremont, CA 94539
Reading Property Info: 34221 Finnigan Ter,Fremont, CA 94555
Reading Property Info: 39109 Guardino Dr #230,Fremont, CA 94538
Reading Property Info: 38524 Athy Ct,Fremont, CA 94536
Reading Property Info: 48920 Crestview Cmn,Fremont, CA 94539
Reading Property Info: 3534 Sequoia Cmn,FREMONT, CA 94536
Reading Property Info: 42253 Camino Santa Barbar

Reading Property Info: 4841 Claremont Park Ct,Fremont, CA 94538
Reading Property Info: 34183 Aberdeen Ter,Fremont, CA 94555
Reading Property Info: 340 Paloma Ter,Fremont, CA 94536
Reading Property Info: 34331 Quartz Ter,FREMONT, CA 94555
Reading Property Info: 48887 Crestview Cmn Unit CM,FREMONT, CA 94539
Reading Property Info: 4434 Ladner St,FREMONT, CA 94538
Reading Property Info: 38627 Cherry Ln #86,Fremont, CA 94536
Reading Property Info: 38756 Huntington Cir,Fremont, CA 94536
Reading Property Info: 1171 Vuelta Olivos,FREMONT, CA 94539
Reading Property Info: 4975 Tenor Ct,Fremont, CA 94538
Reading Property Info: 34332 Portia Ter,FREMONT, CA 94555
Reading Property Info: 39653 Whitecap Way,Fremont, CA 94538
Reading Property Info: 669 Praderia Cir,Fremont, CA 94539
Reading Property Info: 47447 Hoyt St,Fremont, CA 94539
Reading Property Info: 3906 Borgo Cmn,FREMONT, CA 94538
Reading Property Info: 176 Carson Falls Ter,Fremont, CA 94539
Reading Property Info: 37280 Spruce Ter,Fremont, C

,Address,Sold,Sold Date,Bed,Bath,Size,Lot Size,HOA,Year Build,Type
1,"62 Shaniko Cmn,Fremont, CA 94539","$800,000","OCT 1, 2021",3,2,"1,150",0,$310,1987,Condo/Co-op
2,"45552 Cheyenne Pl,Fremont, CA 94539","$1,880,000","OCT 1, 2021",4,2,"1,531","6,300 Sq. Ft.",0,1969,Single Family Residential
3,"1201 Valdez Way,FREMONT, CA 94539","$1,800,000","OCT 1, 2021",3,2,"1,956","6,600 Sq. Ft.",0,1971,Single Family Residential
4,"39762 Blacow Rd,Fremont, CA 94538","$1,150,000","OCT 1, 2021",3,2,"1,585","6,400 Sq. Ft.",0,1961,Single Family Residential
5,"4405 Calypso Ter,Fremont, CA 94555","$1,530,000","OCT 1, 2021",4,2.5,"1,811","5,202 Sq. Ft.",$50,1992,Single Family Residential
...,...,...,...,...,...,...,...,...,...,...
346,"WARM SPRINGS BLVD,FREMONT, CA 94538","$910,384","AUG 20, 2021",2,2,"1,265",0,0,2020,Condo/Co-op
347,"MOWRY AVE,FREMONT, CA 94538","$1,498,537","AUG 2, 2021",0,0,0,"4,519 Sq. Ft.",0,—,Single Family Residential
348,"MOWRY AVE,FREMONT, CA 94538","$1,510,364","JUL 30, 2021",0,0,0,"5,210 Sq. Ft.",0,—,Single Family Residential
349,"WARM SPRINGS BLVD,FREMONT, CA 94538","$2,401,874","AUG 13, 2021",0,0,0,0,0,—,Single Family Residential


In [ ]:
import requests
from bs4 import BeautifulSoup as BS
import json
import numpy as np
import pandas as pd
import csv
import ssl
import urllib.request 
import urllib.parse
import urllib.error
import re
import time
from lxml import etree
import lxml.html as PARSER
from urllib.request import Request, urlopen
req_headers = {
    'accept': '/',
    'accept-encoding': 'gzip, deflate, br',
    'user-agent': 'PostmanRuntime/7.28.4'
}

# For ignoring SSL certificate errors
ctx = ssl.create_default_context()
ctx.check_hostname = False
ctx.verify_mode = ssl.CERT_NONE
domain = "https://www.redfin.com"


# Location
fremont = '6671/CA/Fremont'
losAngeles = '11203/CA/Los-Angeles'
seattle = '16163/WA/Seattle'
newYork = '30749/NY/New-York'



for page in range(1,10):

    # Sold Properties
    soldPropertiesURL = 'https://www.redfin.com/city/' + fremont + '/filter/property-type=house+condo+townhouse,include=sold-3yr/page-' + str(page)
    time.sleep(np.random.lognormal(0,0.01))
    req = Request(soldPropertiesURL, headers={'User-Agent': 'Mozilla/5.0'})
    webpage = urlopen(req).read()
    text = webpage.decode(encoding="utf8", errors="ignore")
    with open("page"+str(page)+".html", "w") as f:
        f.write(text) 
        f.close()

In [ ]:
for page in range(1,10):

    with open("page"+str(page)+".html", "r") as f:
        soup = BS(f, 'html.parser')
    
    bottomV2_list  = soup.findAll('div',attrs={'class':"bottomV2"})
    id_list  = re.findall('href=\"(.*?)\"',str(bottomV2_list))    
    # Construct Single Property URL
    for i in range(len(bottomV2_list)):
        singlePropertyURL = domain + id_list[i]
        time.sleep(3+np.random.lognormal(1,1))
        singlePropertyReq = Request(singlePropertyURL, headers={'User-Agent': 'Mozilla/7.0 (Macintosh; Intel Mac OS X 10_12_6) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/76.0.3809.132 Safari/537.36'})
        singlePropertyWebpage = urlopen(singlePropertyReq).read()
        singlePropertyText = singlePropertyWebpage.decode(encoding="utf8", errors="ignore")
        with open("singlePropertyWebpage"+str(page)+str(i)+".html", "w") as f:
            f.write(singlePropertyText) 
            f.close()

In [1]:
import requests
from bs4 import BeautifulSoup as BS
import json
import numpy as np
import pandas as pd
import csv
import ssl
import urllib.request 
import urllib.parse
import urllib.error
import re
import time
from lxml import etree
import lxml.html as PARSER
from urllib.request import Request, urlopen
addresses_list = []
sold_prices_list = []
bed_list = []
bath_list = []
sqft_list = []
hoa_list = []
yearBuild_list = []
type_list = []
lotSize_list = []
sold_date_main_list = []
def isXpath(content, xpath):
    root = PARSER.fromstring(content)
    if root.xpath(xpath):
        return True
    return False

for page in range(1,10):
    print("page",page)
    with open("page"+str(page)+".html", "r") as f:
        soup = BS(f, 'html.parser')
    
    bottomV2_list  = soup.findAll('div',attrs={'class':"bottomV2"})
    id_list  = re.findall('href=\"(.*?)\"',str(bottomV2_list))
    sold_date_list = soup.find_all("div", attrs={'class':["Pill Pill--sold padding-vert-smallest padding-horiz-smaller font-size-smaller font-weight-bold font-color-white HomeSash margin-top-smallest margin-right-smaller", "Pill Pill--red padding-vert-smallest padding-horiz-smaller font-size-smaller font-weight-bold font-color-white HomeSash margin-top-smallest margin-right-smaller"]})
    
    for i in range(len(sold_date_list)):
        sold_date_list[i] = sold_date_list[i].get_text().replace("SOLD BY REDFIN ","").replace("BOUGHT WITH REDFIN ","").replace("SOLD ","")
        sold_date_list[i] = str(sold_date_list[i])
    
    sold_date_main_list.extend(sold_date_list)
    
        
        
    for i in range(len(bottomV2_list)): 
        print('item',i)
        with open("singlePropertyWebpage"+str(page)+str(i)+".html", "r") as f:
            singlePropertySoup = BS(f, 'html.parser')
        dom = etree.HTML(str(singlePropertySoup))
    
        # Locate element
        address_list = singlePropertySoup.findAll('div',attrs={'class':"street-address"})
        city_list = singlePropertySoup.findAll('div',attrs={'class':"dp-subtext"})
        address = address_list[0].get_text() + city_list[0].get_text()
        print("Reading Property Info: " + address)

        sold_price_list = singlePropertySoup.findAll('div',attrs={'class':"statsValue"})
        bed = dom.xpath('//div[@class="stat-block beds-section"][2]/div/text()')
        bath = dom.xpath('//div[@class="stat-block baths-section"][1]/div/text()')
        sqft = dom.xpath('//div[@class="stat-block sqft-section"][1]/span/text()')
        lotSize = dom.xpath('//span[text() = "Lot Size"]/../span[2]/text()')
        if isXpath(singlePropertySoup.text, '//span[text() = "HOA Dues"]'):
            hoa = dom.xpath('//span[text() = "HOA Dues"]/../span[2]/text()')
        else:
            hoa = 0
        type = dom.xpath('//span[text() = "Style"]/../div/text()')[0]
        yearBuild = dom.xpath('//span[text() = "Year Built"]/../div/text()')[0]
    
        address = address_list[0].get_text() + city_list[0].get_text()
        sold_price = sold_price_list[0].get_text()
        
        # Fix types
        bed = 0 if len(bed)==0 else bed[0]
        bath = 0 if len(bath)==0 else bath[0]
        sqft = 0 if len(sqft)==0 else sqft[0]
        lotSize = 0 if len(lotSize)==0 else lotSize[0]
        if hoa != 0:
            if len(hoa)!=0:
                hoa = hoa[0].replace("/month","")
            else:
                hoa = 0

        # add to main list
        addresses_list.append(address)
        sold_prices_list.append(sold_price)
        bed_list.append(bed)
        bath_list.append(bath)
        sqft_list.append(sqft)
        hoa_list.append(hoa)
        yearBuild_list.append(yearBuild)
        type_list.append(type)
        lotSize_list.append(lotSize)

df = pd.DataFrame(list(zip(addresses_list, sold_prices_list, sold_date_main_list, bed_list, bath_list, sqft_list,
                           lotSize_list, hoa_list, yearBuild_list, type_list)),
                  columns=("Address", "Sold", "Sold Date", "Bed", "Bath", "Size", 
                           "Lot Size", "HOA", "Year Build", "Type"))
#df.index+=1
df.to_csv('Fremont 3 years.csv')
df

page 1
item 0
Reading Property Info: 40200 Antigua Rose Ter,FREMONT, CA 94538
item 1
Reading Property Info: 4777 Canvasback Cmn,Fremont, CA 94555
item 2
Reading Property Info: 2861 Pinnacles Ter,Fremont, CA 94538
item 3
Reading Property Info: 4022 Abbey Ter #104,Fremont, CA 94536
item 4
Reading Property Info: 5696 Pandorea Ter,NEWARK, CA 94560
item 5
Reading Property Info: 38047 Miller Pl,FREMONT, CA 94536
item 6
Reading Property Info: 5417 Diana Cmn,FREMONT, CA 94555
item 7
Reading Property Info: 38780 Tyson Ln Unit 105C,Fremont, CA 94536
item 8
Reading Property Info: 3966 Lake Tahoe Ter,Fremont, CA 94555
item 9
Reading Property Info: 43876 Paso Pino Cmn,FREMONT, CA 94539
item 10
Reading Property Info: 33035 Lake Erie St,FREMONT, CA 94555
item 11
Reading Property Info: 33892 Abercrombie Pl,Fremont, CA 94555
item 12
Reading Property Info: 34160 Via Lucca,Fremont, CA 94555
item 13
Reading Property Info: 180 Yampa Way,FREMONT, CA 94539
item 14
Reading Property Info: 4893 Deep Creek Rd,FR

item 40
Reading Property Info: 4660 Drury Ct,Fremont, CA 94538
page 4
item 0
Reading Property Info: 4640 Sloat Rd,Fremont, CA 94538
item 1
Reading Property Info: 5619 Willkie Pl,Fremont, CA 94538
item 2
Reading Property Info: 4283 Fairwood St,Fremont, CA 94538
item 3
Reading Property Info: 40219 Laiolo Rd,Fremont, CA 94538-2606
item 4
Reading Property Info: 5631 Buchanan Pl,Fremont, CA 94538
item 5
Reading Property Info: 2164 Jackson St,Fremont, CA 94539
item 6
Reading Property Info: 40783 Chiltern Dr,Fremont, CA 94539-3703
item 7
Reading Property Info: 4536 Cerritos Ave,Fremont, CA 94536
item 8
Reading Property Info: 1734 Horner Way,Fremont, CA 94536
item 9
Reading Property Info: 38818 ALTURA,Fremont, CA 94536-4402
item 10
Reading Property Info: 4772 PORTER St,Fremont, CA 94538-2525
item 11
Reading Property Info: 40471 Blanchard St,Fremont, CA 94538
item 12
Reading Property Info: 34272 Newton Pl,Fremont, CA 94555
item 13
Reading Property Info: 37880 Bright Cmn,Fremont, CA 94536
item 1

Reading Property Info: 39219 Guardino Dr #267,FREMONT, CA 94538
item 40
Reading Property Info: 40358 Sundale Dr,Fremont, CA 94538
page 7
item 0
Reading Property Info: 39400 Albany Cmn Unit J,Fremont, CA 94538
item 1
Reading Property Info: 35143 Lancero St,Fremont, CA 94536
item 2
Reading Property Info: 194 Freedom Ct,Fremont, CA 94539
item 3
Reading Property Info: 4681 Bianca Dr,Fremont, CA 94536
item 4
Reading Property Info: 37065 Dutra Way,Fremont, CA 94536
item 5
Reading Property Info: 4018 Central Ave #104,Fremont, CA 94536
item 6
Reading Property Info: 39583 Buena Vista Ter,Fremont, CA 94538
item 7
Reading Property Info: 38623 Cherry Ln #117,FREMONT, CA 94536
item 8
Reading Property Info: 4544 Merced Cmn,Fremont, CA 94536
item 9
Reading Property Info: 150 Branta Cmn,FREMONT, CA 94539
item 10
Reading Property Info: 380 Apache Ct,Fremont, CA 94539
item 11
Reading Property Info: 581 Chantecler Dr,Fremont, CA 94539
item 12
Reading Property Info: 2356 Gomes Rd,Fremont, CA 94539
item 13

,Address,Sold,Sold Date,Bed,Bath,Size,Lot Size,HOA,Year Build,Type
0,"40200 Antigua Rose Ter,FREMONT, CA 94538","$1,375,000","SEP 24, 2021",4,4.5,"1,917","2,127 Sq. Ft.",0,2008,Townhouse
1,"4777 Canvasback Cmn,Fremont, CA 94555","$1,400,000","SEP 16, 2021",3,2.5,"1,607","2,826 Sq. Ft.",0,1988,Single Family Residential
2,"2861 Pinnacles Ter,Fremont, CA 94538","$1,330,000","SEP 10, 2021",4,3.5,"2,093","1,768 Sq. Ft.",0,2013,Single Family Residential
3,"4022 Abbey Ter #104,Fremont, CA 94536","$549,000","SEP 8, 2021",2,1,748,0,0,1986,Condo/Co-op
4,"5696 Pandorea Ter,NEWARK, CA 94560","$1,094,000","SEP 1, 2021",3,3.5,"1,925","1,384 Sq. Ft.",0,2017,Townhouse
...,...,...,...,...,...,...,...,...,...,...
345,"39725 Costa Way,Fremont, CA 94538","$1,299,328","AUG 20, 2020",5,3,"1,767",0,0,1963,Multi-Family (2-4 Unit)
346,"34394 Livorna Ter #33,Fremont, CA 94555","$1,004,939","JAN 16, 2019",2,2.5,"1,395",0,0,1992,Townhouse
347,"3855 Shovler Lake Ct,Fremont, CA 94555","$1,360,681","OCT 25, 2019",3,3,"1,335",0,0,1978,Single Family Residential
348,"41965 Osgood Rd,Fremont, CA 94539","$3,324,684","APR 23, 2019",3,3,"2,208",0,0,1955,Single Family Residential


In [33]:
np.size(addresses_list)

2138